## Bart Finetuner

In [2]:
import torch
import pandas as pd
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW
from rouge_score import rouge_scorer

#gets data from csv
def load_data(csv_file):
    data = pd.read_csv(csv_file)
    questions = data['Question'].tolist()
    answers = data['Answer'].tolist()
    return questions, answers

#tokenises data and converts to integer representations of the token to be used as ids.
def preprocess_data(questions, answers):
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
    input_ids = []
    attention_masks = []

    for question in questions:
        encoded = tokenizer.encode_plus(
            question,
            add_special_tokens=True,
            padding='max_length',
            max_length=256,
            return_tensors='pt'
        )
        input_ids.append(encoded.input_ids)
        attention_masks.append(encoded.attention_mask)

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = tokenizer.batch_encode_plus(
        answers,
        add_special_tokens=True,
        padding='max_length',
        max_length=256,
        return_tensors='pt'
    ).input_ids

    return input_ids, attention_masks, labels

#training function
def train_model(input_ids, attention_masks, labels):
    model = BartForConditionalGeneration.from_pretrained('textgenerator')
    optimizer = AdamW(model.parameters(), lr=1e-5)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    input_ids = input_ids.to(device)
    attention_masks = attention_masks.to(device)
    labels = labels.to(device)

    model.train()
    train_dataset = torch.utils.data.TensorDataset(input_ids, attention_masks, labels)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)

    #loops through dataset 5 times
    for epoch in range(5):
        total_loss = 0
        print(f'Epoch: {epoch + 1}')
        #batches of 8 qa pairs
        for batch in train_loader:
            batch = tuple(t.to(device) for t in batch)
            input_ids, attention_masks, labels = batch

            model.zero_grad()
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_masks,
                labels=labels
            )

            loss = outputs.loss
            total_loss += loss.item()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            print(f'  Batch Loss: {loss.item():.4f}')

        avg_loss = total_loss / len(train_loader)
        print(f'Average Loss: {avg_loss:.4f}')

    return model


#saves model
def save_model(model, save_path):
    model.save_pretrained(save_path)

#generate answers
def generate_answers(questions, model, tokenizer):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    generated_answers = []

    for question in questions:
        encoded = tokenizer.encode_plus(
            question,
            add_special_tokens=True,
            padding='max_length',
            max_length=256,
            return_tensors='pt'
        )
        input_ids = encoded.input_ids.to(device)
        attention_mask = encoded.attention_mask.to(device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
                early_stopping=True
            )

        generated_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_answers.append(generated_answer)

    return generated_answers
#rouge eval
def evaluate_rouge(generated_answers, ground_truth_answers):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(generated_answers, ground_truth_answers)
    return rouge_scores

#main
def main():
    csv_file = 'qadata.csv'
    save_path = 'textgenerator'
    
    questions, answers = load_data(csv_file)
    input_ids, attention_masks, labels = preprocess_data(questions, answers)
    model = train_model(input_ids, attention_masks, labels)
    save_model(model, save_path)
    print('Training complete. Model saved.')

    #eval
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
    generated_answers = generate_answers(questions, model, tokenizer)
    rouge_scores = evaluate_rouge(generated_answers, answers)
    print('Rouge scores:', rouge_scores)

#callable from other script
if __name__ == '__main__':
    main()



/Users/justin/anaconda3/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 1
  Batch Loss: 18.7929
  Batch Loss: 16.6220
  Batch Loss: 15.5640
  Batch Loss: 15.2382
  Batch Loss: 14.5405
  Batch Loss: 14.4614
  Batch Loss: 14.7551
  Batch Loss: 14.0499
  Batch Loss: 13.4992
  Batch Loss: 13.4783
  Batch Loss: 13.0746
  Batch Loss: 12.6784
  Batch Loss: 12.6247
  Batch Loss: 12.3569
  Batch Loss: 12.6336
  Batch Loss: 12.4861
  Batch Loss: 12.2680
  Batch Loss: 12.3079
  Batch Loss: 12.1260
  Batch Loss: 11.9773
  Batch Loss: 12.0187
  Batch Loss: 11.8305
  Batch Loss: 12.0411
  Batch Loss: 11.2976
  Batch Loss: 11.9437
  Batch Loss: 11.5230
  Batch Loss: 11.6720
  Batch Loss: 11.3411
  Batch Loss: 11.3095
  Batch Loss: 11.3996
  Batch Loss: 11.5677
  Batch Loss: 10.9039
  Batch Loss: 10.9198
  Batch Loss: 11.2832
  Batch Loss: 10.7567
  Batch Loss: 10.4492
  Batch Loss: 10.8263
  Batch Loss: 10.7635
  Batch Loss: 10.5747
  Batch Loss: 10.2325
  Batch Loss: 10.6975
  Batch Loss: 10.2802
  Batch Loss: 10.3521
  Batch Loss: 9.9723
  Batch Loss: 10.2158
  

NameError: name 'tokenizer' is not defined

## Rouge Evaluator

In [7]:
import pandas as pd
from transformers import BartTokenizer, BartForConditionalGeneration
from rouge_score import rouge_scorer

#loads
def load_data(csv_file):
    data = pd.read_csv(csv_file)
    questions = data['Question'].tolist()
    answers = data['Answer'].tolist()
    return questions, answers

#generate answer
def generate_answers(questions, model, tokenizer):
    generated_answers = []
    for question in questions:
        input_ids = tokenizer.encode(question, add_special_tokens=True, truncation=True, max_length=512, return_tensors='pt')
        output_ids = model.generate(input_ids=input_ids.to(model.device), max_length=512, num_beams=5)
        generated_answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        generated_answers.append(generated_answer)
    return generated_answers

#main
csv_file = 'rougeeval.csv'
save_path = 'textgenerator'
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained(save_path)

questions, answers = load_data(csv_file)

generated_answers = generate_answers(questions, model, tokenizer)

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

#rouge score calculator
rouge_scores = []
for generated_answer, reference_answer in zip(generated_answers, answers):
    score = scorer.score(generated_answer, reference_answer)
    rouge_scores.append(score)

print('Rouge scores:', rouge_scores)


Rouge scores: [{'rouge1': Score(precision=0.27906976744186046, recall=0.375, fmeasure=0.31999999999999995), 'rouge2': Score(precision=0.047619047619047616, recall=0.06451612903225806, fmeasure=0.0547945205479452), 'rougeL': Score(precision=0.20930232558139536, recall=0.28125, fmeasure=0.24)}, {'rouge1': Score(precision=0.803921568627451, recall=0.8913043478260869, fmeasure=0.845360824742268), 'rouge2': Score(precision=0.72, recall=0.8, fmeasure=0.7578947368421052), 'rougeL': Score(precision=0.7647058823529411, recall=0.8478260869565217, fmeasure=0.8041237113402062)}, {'rouge1': Score(precision=0.5238095238095238, recall=0.7333333333333333, fmeasure=0.611111111111111), 'rouge2': Score(precision=0.34146341463414637, recall=0.4827586206896552, fmeasure=0.4000000000000001), 'rougeL': Score(precision=0.35714285714285715, recall=0.5, fmeasure=0.41666666666666663)}, {'rouge1': Score(precision=0.43137254901960786, recall=0.4888888888888889, fmeasure=0.45833333333333326), 'rouge2': Score(precis

## Human Evaluation

In [2]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration

#load model and tokenizer
save_path = 'textgenerator'
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained(save_path)

#use cpu if gpu is not available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

#generate answers
while True:
    question = input("Enter your question (or 'q' to quit): ")

    if question.lower() == 'q':
        break

    #tokenise and encode question
    inputs = tokenizer.encode_plus(
        question,
        add_special_tokens=True,
        padding='longest',
        max_length=256,
        return_tensors='pt'
    )
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    #generate answer
    with torch.no_grad():
        outputs = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=256,
            num_beams=5,
            num_return_sequences=1,
            early_stopping=True
        )

    #decodes answer
    generated_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("Generated Answer:", generated_answer)


Generated Answer: Swinburne Online offers a wide range of online education services, including degree programs, tutorials, eLearning Advisors, and dedicated online tutors in each of its Melbourne and overseas campuses. Learn more about our online degree programs.
